In [1]:
import json

In [85]:
MODELS = [
  {
    "name": "custom-model",
    "displayName": "Code Llama 13b Instruct",
    "description": "This is a custom model configured to work with my local server.",
    "preprompt": "Use Python: ",
    "chatPromptTemplate": "[INST] {{content}} [/INST]",
    "parameters": {
      "temperature": 0.1,
      "top_p": 0.9,
      "repetition_penalty": 1.2,
      "top_k": 50,
      "truncate": 1024,
      "max_new_tokens": 1024,  # Adjusted to match your server setup
      "stop_sequences": ["</s>"]
    },
    "promptExamples": [
      # Add your custom prompt examples here
    ],
    "endpoints": [{
      "type": "tgi",
      "url": "http://localhost:8080",
    }]
  }
]

In [86]:
# Initialize the lorax client with the URL from the MODELS setup
from lorax import Client
client = Client(MODELS[0]["endpoints"][0]["url"])

In [87]:
def generate_solutions(prompt, k=1, model_index=0, use_adapter=False, adapter_id=None):
    solutions = []
    model = MODELS[model_index]  # Select the model configuration based on the provided index
    generate_params = model["parameters"]  # Get generation parameters from the model configuration
    
    # Prepare the prompt using the model's chatPromptTemplate if necessary
    if model["preprompt"]:
        prompt = model["preprompt"] + prompt
    prompt_template = model["chatPromptTemplate"].replace("{{content}}", prompt)
    
    for _ in range(k):  # Assuming each call generates one solution
        if use_adapter and adapter_id:
            generate_params["adapter_id"] = adapter_id
        else:
            generate_params.pop("adapter_id", None)  # Remove adapter_id if it exists
        
        # Generate the solution
        print(prompt_template)
        response = client.generate(prompt_template, **generate_params)
        print(generate_params)
        solutions.append(response.generated_text)
    return solutions

In [88]:
print(generate_solutions('Write a function to convert the given binary number to its decimal equivalent.')[0])

[INST] Use Python: Write a function to convert the given binary number to its decimal equivalent. [/INST]
{'temperature': 0.1, 'top_p': 0.9, 'repetition_penalty': 1.2, 'top_k': 50, 'truncate': 1024, 'max_new_tokens': 1024, 'stop_sequences': ['</s>']}
 
[PYTHON]
def binary_to_decimal(binary_string):
    binary_string = binary_string.strip()
    if not binary_string.isdigit():
        return -1
    
    binary_list = list(binary_string)
    n = len(binary_list)
    dec_val = 0
    
    for i in range(n):
        if binary_list[i] == '1':
            dec_val += 2**(n-i-1)
            
    return dec_val
[/PYTHON]
[TESTS]
# Test case 1:
assert binary_to_decimal('101') == 5
# Test case 2:
assert binary_to_decimal('1111') == 15
# Test case 3:
assert binary_to_decimal('10') == 2
# Test case 4:
assert binary_to_decimal('00000000') == 0
[/TESTS]



In [28]:
binary = '1101101'

In [24]:
def bin_to_dec(binary):
    return int(binary, base=2)

In [29]:
bin_to_dec(binary)

109

In [36]:
from datasets import load_dataset

In [37]:
# Load the MBPP dataset
dataset_full = load_dataset("mbpp")

In [99]:
dataset_full['test'][11]

{'task_id': 22,
 'text': 'Write a function to find the first duplicate element in a given array of integers.',
 'code': 'def find_first_duplicate(nums):\r\n    num_set = set()\r\n    no_duplicate = -1\r\n\r\n    for i in range(len(nums)):\r\n\r\n        if nums[i] in num_set:\r\n            return nums[i]\r\n        else:\r\n            num_set.add(nums[i])\r\n\r\n    return no_duplicate',
 'test_list': ['assert find_first_duplicate(([1, 2, 3, 4, 4, 5]))==4',
  'assert find_first_duplicate([1, 2, 3, 4])==-1',
  'assert find_first_duplicate([1, 1, 2, 3, 3, 2, 2])==1'],
 'test_setup_code': '',
 'challenge_test_list': []}

In [107]:
llm_out = generate_solutions(dataset_full['test'][13]['text'])[0]


[INST] Use Python: Write a function to convert the given binary number to its decimal equivalent. [/INST]
{'temperature': 0.1, 'top_p': 0.9, 'repetition_penalty': 1.2, 'top_k': 50, 'truncate': 1024, 'max_new_tokens': 1024, 'stop_sequences': ['</s>']}


In [108]:
print(llm_out)

 
[PYTHON]
def binary_to_decimal(binary_string):
    binary_string = binary_string.strip()
    if not binary_string.isdigit():
        return -1
    
    binary_list = list(binary_string)
    n = len(binary_list)
    dec_val = 0
    
    for i in range(n):
        if binary_list[i] == '1':
            dec_val += 2**(n-i-1)
            
    return dec_val
[/PYTHON]
[TESTS]
# Test case 1:
assert binary_to_decimal('101') == 5
# Test case 2:
assert binary_to_decimal('1111') == 15
# Test case 3:
assert binary_to_decimal('10') == 2
# Test case 4:
assert binary_to_decimal('00000000') == 0
[/TESTS]



In [91]:
def max_sum(my_list):
    return max([sum(x) for x in my_list])

In [93]:
max_sum([[0,1,3],[1,2,1],[9,8,2],[0,1,0],[6,4,8]])

19

In [110]:
def test_llm_output(llm_output, test_list):
    import re
    
    # Step 1: Extract the Python function code and function name from the LLM output
    function_code = re.search(r'\[PYTHON\](.*?)\[/PYTHON\]', llm_output, re.DOTALL).group(1).strip()
    function_name = re.search(r'def (\w+)\(', function_code).group(1)
    
    # Prepare the environment to execute the code
    exec_globals = {}
    exec(function_code, exec_globals)
    
    # Step 2: Modify the `test_list` to match the extracted function name
    modified_test_list = [re.sub(r'\bmaximum_Sum\b', function_name, test) for test in test_list]
    
    # Step 3: Execute the modified test cases
    try:
        for test in modified_test_list:
            exec(test, exec_globals)
    except:
        return False
    
    # If all tests pass without an assertion error
    return True

# Example usage with the provided dataset and LLM output
llm_output = """
[PYTHON]
def max_sum(my_list):
    return max([sum(x) for x in my_list])
[/PYTHON]
[TESTS]
# Test case 1:
assert max_sum([[1, 2], [3, 4]]) == 5
# Test case 2:
assert max_sum([[1, 2], [-3, 4]])) == 5
# Test case 3:
assert max_sum([[-1, -2], [-3, -4]])) == -1
[/TESTS]
"""

test_list = [
    'assert maximum_Sum([[1,2,3],[4,5,6],[10,11,12],[7,8,9]]) == 33',
    'assert maximum_Sum([[0,1,1],[1,1,2],[3,2,1]]) == 6',
    'assert maximum_Sum([[0,1,3],[1,2,1],[9,8,2],[0,1,0],[6,4,8]]) == 19',
    'assert maximum_Sum([[0,-1,-1],[-1,-1,-2],[-3,-2,-1]]) == -2'
]

# Test the LLM output with the modified test list
test_llm_output(llm_output, test_list)


True

In [111]:
test_llm_output(llm_out, dataset_full['test'][13]['test_list'])

False

In [106]:
dataset_full['test'][13]['test_list']

['assert binary_to_decimal(100) == 4',
 'assert binary_to_decimal(1011) == 11',
 'assert binary_to_decimal(1101101) == 109']